# Experiments

<br>

### Imports

In [1]:
import numpy as np
import pandas as pd

import copy
from tqdm import tqdm

from evtdemand import suite

In [2]:
from IPython.display import JSON

In [49]:
default_params = {
    'data_dir': '../data',
    'model_1': 'sklearn.ensemble.RandomForestRegressor', 
    'model_2': 'sklearn.ensemble.RandomForestRegressor',
    'model_1_kwargs': {},
    'model_2_kwargs': {},
    'data_kwargs': {
        'real_power_sub_dir': 'real_power',
        'weather_sub_dir': 'weather',
        'real_power_time_period': '_pre_august',
        'real_power_site': 'Staplegrove_CB905',
        'weather_grid_point': 'staplegrove_1',
        'weather_interpolate_method': 'interpolate'
    },
    'y1_col': 'value_max',
    'y2_col': 'value_min',
    'split_kwargs': {
        'n_splits': 5, 
        'shuffle': False
    },
    'cols_subset': ['value', 'temperature', 'solar_irradiance', 'pressure',
                    'spec_humidity', 'hour', 'windspeed_north', 'windspeed_east',
                    'doy', 'speed', 'direction', 'weekend', 'hcdh'],#, 'prev_month_max_avg',
                    #'prev_month_max_max', 'prev_month_min_avg', 'prev_month_min_min'],
    'features_kwargs': {
        'features': ['temporal', 'dir_speed', 'hcdh']
    }
}

In [4]:
%%time

model_suite, error_metrics, df_pred, input_data = suite.run_parameterised_model(**default_params)

error_metrics

Wall time: 8min 2s


{'y1_rmse': 10.460005773284186,
 'y2_rmse': 11.49911414894963,
 'combined_rmse': 15.590660164682564,
 'skill_score': 0.5271113043079159}

In [6]:
%%time 

# comparing the two wind speed encodings

model_suite, error_metrics, df_pred, input_data = suite.run_parameterised_model(
    data_dir=default_params['data_dir'], 
    split_kwargs=default_params['split_kwargs'],
    cols_subset=['value', 'temperature', 'solar_irradiance', 'pressure',
                 'spec_humidity', 'hour', 'doy', 'speed', 'direction', 'weekend', 'hcdh'])
vectors_error = error_metrics['skill_score']

model_suite, error_metrics, df_pred, input_data = suite.run_parameterised_model(
    data_dir=default_params['data_dir'], 
    split_kwargs=default_params['split_kwargs'],
    cols_subset=['value', 'temperature', 'solar_irradiance', 'pressure', 'spec_humidity', 
                 'hour', 'windspeed_north', 'windspeed_east', 'doy', 'weekend', 'hcdh'])
speed_dir_error = error_metrics['skill_score']


vectors_error, speed_dir_error

Wall time: 12min 56s


(0.5255203515257487, 0.5250321153868784)

In [13]:
df_results = pd.DataFrame()
all_input_data = []
test_splits = list(range(2, 7))

for run_id, test_split in tqdm(enumerate(test_splits), total=len(test_splits)):    
    model_suite, error_metrics, df_pred, input_data = suite.run_parameterised_model(
        data_dir=default_params['data_dir'], 
        split_kwargs={'n_splits': test_split, 'shuffle': False}
    )
    
    df_results = df_results.append(pd.DataFrame([error_metrics], index=[run_id]))
    all_input_data += [input_data]

df_results.index.name = 'run_id'
    
df_results

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [23:48<00:00, 285.74s/it]


,y1_rmse,y2_rmse,combined_rmse,skill_score
run_id,,,,
0,17.617496,18.559083,25.595969,0.530863
1,14.243697,14.955248,20.666098,0.529086
2,11.765296,12.935423,17.517853,0.529014
3,10.413627,11.496951,15.555175,0.525496
4,9.641394,10.463222,14.278404,0.536105


In [59]:
%%time

params = copy.deepcopy(default_params)

params['model_1'] = 'lightgbm.LGBMRegressor'
params['model_2'] = 'lightgbm.LGBMRegressor'
params['model_1_kwargs'] = {
    'num_leaves': 50,
    'n_estimators': 100,
    'reg_lambda': 0.1
}
params['model_2_kwargs'] = {
    'num_leaves': 50,
    'n_estimators': 100,
    'reg_lambda': 0.1
}

model_suite, error_metrics, df_pred, input_data = suite.run_parameterised_model(**params)

error_metrics

Wall time: 11.9 s


{'y1_rmse': 10.389345154377155,
 'y2_rmse': 11.177505324655117,
 'combined_rmse': 15.305702367282791,
 'skill_score': 0.5152324587016828}